# Find two stocks with high correlation

From [Find Highly Correlated Stocks with Python!](https://towardsdatascience.com/find-highly-correlated-stocks-with-python-77eba4fd061b)

In [1]:
# Uncomment the following lines to install the required packages
# !pip install yfinance
# !pip install pandas_datareader
# !pip install yahoo-fin

In [1]:
import numpy as np
import pandas as pd

import warnings
from pandas_datareader import data as pdr

# to override deprecations in pandas-datareader
import yfinance as yf

# To set up the start/end dates for the prices
import datetime as dt

# to quickly get access to a list of the tickers in different indices
from yahoo_fin import stock_info as si

# Settings
pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")
yf.pdr_override()

In [2]:
# Set up the start/end dates for the prices
num_of_years = 1
start = dt.date.today() - dt.timedelta(days = int(365.25*num_of_years))
end = dt.date.today()

# Get the list of tickers in the Dow Jones index (others: tickers_dow, 
# tickers_ftse100, tickers_ftse250, tickers_ibovespa, tickers_nasdaq, 
# tickers_nifty50, tickers_niftybank, tickers_other, tickers_sp500)
tickers = si.tickers_dow()

Since we are focusing on the correlation between the movement of stocks, we can concentrate on the Adjusted Close column and then create a new Pandas DataFrame with the base-10 logarithm of the daily percentage change. With the Pandas method .corr(), we can create a correlation matrix of the new dataframe.

In [3]:
dataset = pdr.get_data_yahoo(tickers, start, end)['Adj Close']
dataset.info()

[*********************100%***********************]  30 of 30 completed
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2022-04-19 to 2023-04-18
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    251 non-null    float64
 1   AMGN    251 non-null    float64
 2   AXP     251 non-null    float64
 3   BA      251 non-null    float64
 4   CAT     251 non-null    float64
 5   CRM     251 non-null    float64
 6   CSCO    251 non-null    float64
 7   CVX     251 non-null    float64
 8   DIS     251 non-null    float64
 9   DOW     251 non-null    float64
 10  GS      251 non-null    float64
 11  HD      251 non-null    float64
 12  HON     251 non-null    float64
 13  IBM     251 non-null    float64
 14  INTC    251 non-null    float64
 15  JNJ     251 non-null    float64
 16  JPM     251 non-null    float64
 17  KO      251 non-null    float64
 18  MCD     251 non-null    float64
 19  MMM     251 non-null   

In [4]:
stocks_returns = np.log(dataset/dataset.shift(1))

print('\nCorrelation Matrix')
corr_matrix = stocks_returns.corr()
print (corr_matrix)

[*********************100%***********************]  30 of 30 completed

Correlation Matrix
          AAPL      AMGN       AXP        BA       CAT       CRM      CSCO   
AAPL  1.000000  0.294644  0.672386  0.511208  0.463039  0.637644  0.626951  \
AMGN  0.294644  1.000000  0.257096  0.187682  0.275520  0.130206  0.284976   
AXP   0.672386  0.257096  1.000000  0.529201  0.606085  0.545662  0.531807   
BA    0.511208  0.187682  0.529201  1.000000  0.512625  0.416603  0.386590   
CAT   0.463039  0.275520  0.606085  0.512625  1.000000  0.391233  0.423777   
CRM   0.637644  0.130206  0.545662  0.416603  0.391233  1.000000  0.453316   
CSCO  0.626951  0.284976  0.531807  0.386590  0.423777  0.453316  1.000000   
CVX   0.405798  0.125840  0.376219  0.350439  0.630791  0.364749  0.317381   
DIS   0.645932  0.225277  0.633894  0.500348  0.491989  0.585753  0.457082   
DOW   0.504112  0.258791  0.604363  0.479214  0.698278  0.414369  0.436802   
GS    0.576477  0.245254  0.668158  0.530304  0.624

The correlation matrix includes redundant pairs such as AAPL to AAPL or a pair showing up twice (AAPL to MSFT and MSFT to AAPL). We can drop these and rank the dataframe to get the top absolute correlations with the functions in the gist down below.

In [5]:
def get_redundant_pairs(df):
    """drop the correlations between two of the same stocks

    Args:
        df (pd.PandasDataframe): dataframe table of the stocks

    Returns:
        pairs_to_drop (set): set of the pairs of stocks to drop
    """
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df):
    """Drop the correlations between two of the same stocks

    Args:
        df (pd.PandasDataframe): dataframe table of the stocks

    Returns:
        au_corr (pd.PandasDataframe): dataframe table of the stocks, with the 
            highest correlations among the inputted list of tickers
    """
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr

print("\nTop Absolute Correlations")
print(get_top_abs_correlations(stocks_returns))


Top Absolute Correlations
AAPL  MSFT    0.789422
GS    JPM     0.758890
KO    PG      0.734050
AXP   V       0.717266
AAPL  V       0.707470
CAT   DOW     0.698278
KO    MCD     0.691866
HON   MMM     0.691244
AAPL  INTC    0.686196
CAT   HON     0.682448
AAPL  NKE     0.679775
CRM   MSFT    0.674563
AAPL  AXP     0.672386
AXP   GS      0.668158
INTC  MSFT    0.665472
DOW   GS      0.660529
GS    HON     0.648189
AXP   JPM     0.647867
AAPL  DIS     0.645932
HD    MSFT    0.644165
      NKE     0.642576
AAPL  CRM     0.637644
NKE   V       0.636720
AXP   DIS     0.633894
CAT   CVX     0.630791
HD    HON     0.627026
AAPL  CSCO    0.626951
MSFT  V       0.626744
AAPL  HD      0.625663
MSFT  NKE     0.624983
MCD   PG      0.624641
JNJ   MRK     0.624404
CAT   GS      0.624158
DIS   MSFT    0.623319
      V       0.622350
      GS      0.620934
HON   JPM     0.619896
CAT   JPM     0.619315
DOW   HON     0.619108
GS    V       0.613938
DOW   JPM     0.609974
CSCO  HON     0.606595
AXP   N

To prove these results we can use [TradingView](https://www.tradingview.com/), to compare the stocks. In our case we have that Apple and Microsoft have an high correlation, maybe because they operate in the same type of market, i.e. technology.